In [1]:
import pandas as pd

import sys
sys.path.append('/Users/basileroth/Desktop/Code/offseason_greenlobby/')
from offseason_greenlobby.config import color_16
import numpy as np
from offseason_greenlobby.utils.utils import get_surrounding_words,extract_until_period,clean_text,get_gp_senat
# define subplot grid
import matplotlib.pyplot as plt 
import re


In [2]:
LIST_COLS = ["IDEES","DESCRIPTION","NOM_AMENDEMENT","EXPOSE_SOMMAIRE",'article_courte',
       'article', 'type_amendement', 'groupe_politique_ref',
       'groupe_politique', 'groupe_politique_nom', 'type_auteur', 'acteur_ref',
       'acteur_nom', 'acteur_prenom', 'acteur_denom']

# CONSTANTES 

In [3]:
ARTICLE = 10

In [4]:
IDEE = "Ne pas interdire les bouteilles en plastique."

In [5]:
DESCRIPTION = "Les amendements visant à assouplir la législation sur les bouteilles en plastique : qu'il s'agisse de repousser la date de leur interdiction, d'exclure certains types de bouteilles du champ d'application, ou encore de conditionner l'interdiction à la réalisation d'une étude d'impact"

# Assemblée nationale

In [6]:
df_an = pd.read_csv("/Users/basileroth/Desktop/Code/offseason_greenlobby/data/processed/AGEC/AN/df_an.csv")
df_an.dropna(subset=["expose_sommaire"],inplace=True)
df_an["IDEES"] = IDEE
df_an["DESCRIPTION"] = DESCRIPTION
df_an["CHAMBRE"] = "AN"

df_an.rename(columns={
    "acteur_denom":"auteur",
    "expose_sommaire":"EXPOSE_SOMMAIRE",
    "nom_du_fichier":"NOM_AMENDEMENT"
},inplace=True)

df_an.rename(columns={
    "groupe_politique":"groupe_politique_abv",
},inplace=True)

df_an.rename(columns={
    "groupe_politique_nom":"groupe_politique",
},inplace=True)

df_an = df_an[df_an.article == ARTICLE]

df_an.shape

(499, 17)

### Sénat

In [7]:
df_senat_total = pd.read_csv("/Users/basileroth/Desktop/Code/offseason_greenlobby/data/processed/AGEC/SENAT/jeu_complet_2018-2019_728.csv", 
   encoding='ISO-8859-1',
   skiprows=1,
   sep='\t')

print("Senat : ",df_senat_total.shape[0])

df_senat_cm = pd.read_csv("/Users/basileroth/Desktop/Code/offseason_greenlobby/data/processed/AGEC/SENAT/jeu_complet_commission_2018-2019_660.csv", 
   encoding='ISO-8859-1',
   skiprows=1,
   sep='\t')

print("Senat comissions: ",df_senat_cm.shape[0])

df_senat_total = pd.concat([df_senat_total,df_senat_cm],axis=0)

df_senat_total.dropna(subset=['Objet '],inplace=True)

df_senat_total['Dispositif_c'] = df_senat_total['Dispositif '].apply(lambda x : clean_text(x))
df_senat_total['Objet_c'] = df_senat_total['Objet '].apply(lambda x : clean_text(x))


df_senat_total['article'] = df_senat_total['Subdivision '].apply(lambda x : ''.join(re.findall(r'\d+', x)))
df_senat_total.loc[df_senat_total['article'] == '',"article"] = 0
df_senat_total['article'] = df_senat_total['article'].apply(lambda x : int(x))

df_senat_total["IDEES"] = IDEE
df_senat_total["DESCRIPTION"] = DESCRIPTION

df_senat_total.rename(columns={
    "Auteur ":"auteur",
    "Objet_c":"EXPOSE_SOMMAIRE",
    "Numéro ":"NOM_AMENDEMENT"
},inplace=True)

df_senat_total["CHAMBRE"] = "SENAT"
# df_senat_total = df_senat_total[df_senat_total.article == ARTICLE]
df_senat_total['groupe_politique'] = df_senat_total['Fiche Sénateur'].astype(str).apply(lambda x : get_gp_senat( "https://"+x[2:]) if len(x) > 5 else "Inconnu")

df_senat_total.shape

Senat :  711
Senat comissions:  537
https://www.senat.fr/senfic/ferat_francoise01025n.html
Membre du  Groupe Union
Centriste
https://www.senat.fr/senfic/ferat_francoise01025n.html
Membre du  Groupe Union
Centriste
https://www.senat.fr/senfic/brisson_max19621t.html
Membre du groupe
Les Républicains
https://www.senat.fr/senfic/brisson_max19621t.html
Membre du groupe
Les Républicains
https://www.senat.fr/senfic/brisson_max19621t.html
Membre du groupe
Les Républicains
https://www.senat.fr/senfic/brisson_max19621t.html
Membre du groupe
Les Républicains
https://www.senat.fr/senfic/darcos_laure19693l.html
Membre du groupe
Les Indépendants - République et Territoires
https://www.senat.fr/senfic/longeot_jean_francois14219q.html
Membre du groupe
Union Centriste
https://www.senat.fr/senfic/longeot_jean_francois14219q.html
Membre du groupe
Union Centriste
https://www.senat.fr/senfic/longeot_jean_francois14219q.html
Membre du groupe
Union Centriste
https://www.senat.fr/senfic/vermeillet_sylvie19479

(1174, 20)

In [ ]:
# df_senat_total.to_csv("/Users/basileroth/Desktop/Code/offseason_greenlobby/data/processed/AGEC/SENAT/df_senat.csv")

# Concat SENAT + AN

In [9]:
pd.concat([
    df_senat_total[["IDEES","DESCRIPTION","NOM_AMENDEMENT","EXPOSE_SOMMAIRE","auteur","groupe_politique","CHAMBRE"]],
    df_an[["IDEES","DESCRIPTION","NOM_AMENDEMENT","EXPOSE_SOMMAIRE","auteur","groupe_politique","CHAMBRE"]],
]).to_csv("../../data/processed/AGEC/df_art"+str(ARTICLE)+".csv")